In [1]:
import sys; sys.path.insert(0, '..') 

In [2]:
import numpy as np
import torch
from preprocess.dataset import get_MNIST,get_dataset,get_handler
from models.model import Model
from al_methods.random_sampling import RandomSampling
from ssl_methods.semi_fixmatch import fixmatch
import models
from torchvision import transforms
from framework.framework1 import Framework1,Strategy1,Strategy2
from copy import deepcopy
import torch.nn as nn
import time

In [3]:
# in the main 
X_tr, Y_tr, X_te, Y_te = get_dataset("Mnist", "./datasets")

In [4]:
# in the main 
if type(X_tr) is list:
    X_tr = np.array(X_tr)
    Y_tr = torch.tensor(np.array(Y_tr))
    X_te = np.array(X_te)
    Y_te = torch.tensor(np.array(Y_te))

if type(X_tr[0]) is not np.ndarray:
    X_tr = X_tr.numpy()
    X_te = X_te.numpy()
    

In [5]:
np.shape(X_tr)[1:]

(28, 28)

In [6]:
# in the main
n_pool = len(Y_tr)
n_test = len(Y_te)

In [7]:
#in the main
handler = get_handler("mnist")

In [8]:
# main or framewrok to see
nEnd=100 # total number to query 
nQuery=1 # nombre of points to query in batch 
nStart=1 # nbre of points to start
NUM_INIT_LB = int(nStart*n_pool/100)
NUM_QUERY = int(nQuery*n_pool/100) if nStart!= 100 else 0
NUM_ROUND = int((int(nEnd*n_pool/100) - NUM_INIT_LB)/ NUM_QUERY) if nStart!= 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd*n_pool/100) - NUM_INIT_LB)% NUM_QUERY != 0:
        NUM_ROUND += 1

In [9]:
print(NUM_INIT_LB,NUM_ROUND,NUM_QUERY)

600 99 600


In [10]:
model_name = sorted(name for name in models.__dict__ 
                    if callable(models.__dict__[name]))

In [11]:
model_name

[]

In [12]:
model=Model('resnet50').get_model()

In [13]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
# net = models.__dict__['ResNet18'](n_class=10)

In [15]:
# net

In [16]:
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Conversion en 3 canaux
model.fc = torch.nn.Linear(2048, 10)  # Modifier la couche de classification pour 10 classes

In [17]:
# net.feature_extractor.conv1=torch.nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [18]:
# net

In [19]:
# in the main file
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_lb

array([False, False, False, ..., False, False, False])

In [20]:
# in the main file
idxs_tmp = np.arange(n_pool)
idxs_tmp

array([    0,     1,     2, ..., 59997, 59998, 59999])

In [21]:
np.random.shuffle(idxs_tmp)
idxs_tmp

array([35977,  9777,   920, ..., 31196, 37159, 20497])

In [22]:
# in the main file
idxs_lb[idxs_tmp[:NUM_INIT_LB]] = True
idxs_lb

array([False, False, False, ..., False, False, False])

### Args

in the main file 

In [23]:
args_pool = {'mnist':
                { 
                 'n_class':10,
                 'channels':1,
                 'size': 28,
                 'transform_tr': transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.1307,), 'std': (0.3081,)},
                },
            
        }

In [24]:
class Args:
    def __init__(self,n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize):
        self.n_class=n_class
        self.img_size=img_size
        self.channels=channels
        self.transform_tr=transform_tr
        self.transform_te=transform_te
        self.loader_tr_args=loader_tr_args
        self.loader_te_args=loader_te_args
        self.normalize=normalize
        self.dataset='mnist'
        self.save_path='./save'
        self.model='ResNet50'
        self.lr=0.1
        self.schedule = [20, 40]
        self.momentum=0.9
        self.gammas=[0.1,0.1]
        self.strategy='framwork1'
        self.optimizer='SGD'
        self.save_model=False

In [25]:
dataset_args = args_pool["mnist"]
n_class = dataset_args['n_class']
img_size = dataset_args['size']
channels = dataset_args['channels']
transform_tr = dataset_args['transform_tr']
transform_te = dataset_args['transform_te']
loader_tr_args = dataset_args['loader_tr_args']
loader_te_args = dataset_args['loader_te_args']
normalize = dataset_args['normalize']

In [26]:
args=Args(n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize)

### Framework setting

In [27]:
# Create instances of strategies
random_strategy = RandomSampling(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)
fixmatch_strategy = fixmatch(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)

In [28]:
# Create an instance of Framework1
framework_1 = Framework1(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args, random_strategy, fixmatch_strategy)

In [29]:
al_method=framework_1.method1(random_strategy)

In [30]:
ssl_method=framework_1.method2(fixmatch_strategy)

In [31]:
al_method.train(2e-3,10)

Let's use 1 GPUs!
[Batch=000] [Loss=2.60]

==>>[2023-08-10 13:28:09] [Epoch=000/010] [framwork1 Need: 00:00:00] [LR=0.1000] [Best : Test Accuracy=0.00, Error=1.00]
[Batch=000] [Loss=9.73]

==>>[2023-08-10 13:28:11] [Epoch=001/010] [framwork1 Need: 00:00:28] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=3.09]

==>>[2023-08-10 13:28:12] [Epoch=002/010] [framwork1 Need: 00:00:19] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=4.74]

==>>[2023-08-10 13:28:14] [Epoch=003/010] [framwork1 Need: 00:00:15] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=2.29]

==>>[2023-08-10 13:28:16] [Epoch=004/010] [framwork1 Need: 00:00:12] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=2.14]

==>>[2023-08-10 13:28:17] [Epoch=005/010] [framwork1 Need: 00:00:09] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=2.36]

==>>[2023-08-10 13:28:19] [Epoch=006/010] [framwork1 Need: 00:00:07] [LR=0.1000] 

0.1555

In [32]:
test_acc=al_method.predict(X_te,Y_te)

In [33]:
test_acc

0.0974

In [34]:
acc = np.zeros(NUM_ROUND+1)
acc[0] = test_acc

In [35]:
# for rd in range(1, NUM_ROUND+1):

#     if rd%2==0:
#         # Al_methods
#         print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
#         labeled = len(np.arange(n_pool)[idxs_lb])
#         if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
#             NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
#         # query
#         ts = time.time()
#         output = al_method.query(NUM_QUERY)
#         q_idxs = output
#         idxs_lb[q_idxs] = True
#         te = time.time()
#         tp = te - ts
        
#         # update
#         al_method.update(idxs_lb)
#         best_test_acc = al_method.train(alpha=2e-3, n_epoch=10)

#         t_iter = time.time() - ts
        
#         # round accuracy
#         # test_acc = strategy.predict(X_te, Y_te)
#         acc[rd] = best_test_acc
#     else:
#         #SSL methods
        
#         print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
#         labeled = len(np.arange(n_pool)[idxs_lb])
#         if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
#             NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
#         # query
#         ts = time.time()
#         output = ssl_method.query(NUM_QUERY)
#         q_idxs = output
#         idxs_lb[q_idxs] = True
#         te = time.time()
#         tp = te - ts
        
#         # update
#         ssl_method.update(idxs_lb)
#         best_test_acc = ssl_method.train(alpha=2e-3, n_epoch=10)

#         t_iter = time.time() - ts
        
#         # round accuracy
#         # test_acc = strategy.predict(X_te, Y_te)
#         acc[rd] = best_test_acc


Round 1/99
Let's use 1 GPUs!
[Batch=000] [Loss=2.63]

==>>[2023-08-10 13:28:32] [Epoch=000/010] [framwork1 Need: 00:00:00] [LR=0.1000] [Best : Test Accuracy=0.00, Error=1.00]
[Batch=000] [Loss=9.82]

==>>[2023-08-10 13:28:38] [Epoch=001/010] [framwork1 Need: 00:00:55] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=3.00]

==>>[2023-08-10 13:28:44] [Epoch=002/010] [framwork1 Need: 00:00:49] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=2.22]

==>>[2023-08-10 13:28:50] [Epoch=003/010] [framwork1 Need: 00:00:42] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=2.15]

==>>[2023-08-10 13:28:56] [Epoch=004/010] [framwork1 Need: 00:00:36] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=2.11]

==>>[2023-08-10 13:29:02] [Epoch=005/010] [framwork1 Need: 00:00:30] [LR=0.1000] [Best : Test Accuracy=0.10, Error=0.90]
[Batch=000] [Loss=1.99]

==>>[2023-08-10 13:29:08] [Epoch=006/010] [framwork1 Need: 00:00:24] [

In [ ]:
acc